In [1]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [2]:
from transformers import (
    BertTokenizer,
    BertModel,
    TFBertModel,
    BertConfig,
    AutoTokenizer,
    AutoModel,
)

In [3]:
import pandas as pd
import numpy as np

In [4]:
import jsonlines
import json

In [5]:
import ast

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
DATA_PATH = '/content/drive/MyDrive/w266_final_project/data/GoodNewsEveryone'

In [8]:
DATA_PATH_LOCAL = './data/GoodNewsEveryone'

In [9]:
# Data needs to be manually partitioned
data = []
with jsonlines.open(f'{DATA_PATH}/gne-release-v1.0.jsonl', 'r') as reader:
    for item in reader:
        data.append(item)

In [10]:
len(data)

5000

In [11]:
data[0].keys()

dict_keys(['id', 'headline', 'url', 'meta', 'annotations'])

In [12]:
df = pd.read_csv(f'{DATA_PATH}/gne-release-v1.0.tsv', sep='\t')
df

,id,headline,url,bias_horizontal,bias_vertical,country,source,dominant_emotion,intensity,cause,experiencer,target,cue,other_emotions,reader_emotions
0,7d3fe468,Cops in One Village Have Been Convicted of 70 ...,http://tracking.feedpress.it/link/9499/12677255,-5.0,46.0,US,ProPublica,anger,medium,"[[""cops in one village have been convicted of ...","[[""cops""]]","[[""cops in one village have been convicted of ...","[[""convicted""]]",annoyance,annoyance
1,86693d59,DIY penis enlargements are a 'nationwide probl...,https://www.dailymail.co.uk/news/article-69149...,13.0,19.0,UK,dailymail,negative_surprise,medium,"[[""diy penis enlargements are a 'nationwide pr...","[[""papua new guinea""]]","[[""diy penis enlargements are a 'nationwide pr...","[[""problem""]]",negative_surprise,no_i_did_not_feel_any_emotion
2,0fb40e90,Dam breaking: New Epstein accuser comes forward,https://hotair.com/archives/ed-morrissey/2019/...,NaN,NaN,US,HotAir,anger,medium,"[[""new epstein accuser comes forward""]]",[[]],"[[""new epstein accuser comes forward""]]","[[""accuser""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
3,fa7750d6,David Beckham gets six-month driving ban for u...,https://www.theguardian.com/football/2019/may/...,-6.0,48.0,UK,TheGuardian,negative_surprise,medium,"[[""gets six-month driving ban for using phone ...","[[""david beckham""]]","[[""driving ban""]]","[[""ban""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
4,695ea7a2,Dead sea turtle found with spear through head ...,https://www.foxnews.com/science/sea-turtle-spe...,27.0,20.0,US,FoxNews,sadness,medium,"[[""dead sea turtle found with spear through he...","[[""florida national park""]]","[[""dead sea turtle found with spear through he...","[[""dead""]]",no_other_additional_emotions_are_expressed,sadness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3cb09e5b,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,https://twitchy.com/samj-3930/2019/06/16/now-d...,29.0,14.0,US,Twitchy,negative_surprise,medium,"[[""on her gun violence claim and only makes he...","[[""ilhan omar""]]","[[""ilhan omar""]]","[[""violence"",""doubles down""]]",negative_surprise,no_i_did_not_feel_any_emotion
4996,86f400a6,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,https://www.thedailybeast.com/in-hong-kong-tri...,-21.0,41.0,US,dailybeast,negative_surprise,medium,"[[""‘triad’ thugs use clubs to punish hong kong...",[[]],"[[""‘triad’ thugs use clubs to punish hong kong...","[[""punish""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
4997,99ca38ca,'For real': High school football season begins...,https://www.theadvocate.com/baton_rouge/sports...,-23.0,40.0,US,TheAdvocate,positive_anticipation_including_optimism,medium,"[[""season begins this week with expectations, ...","[[""high school football""]]","[[""season begins this week with expectations, ...","[[""excitement""]]",positive_surprise,no_i_did_not_feel_any_emotion
4998,f4ae5a03,100 years on: the picture that changed our vie...,https://www.theguardian.com/science/2019/may/1...,-6.0,48.0,UK,TheGuardian,positive_surprise,medium,"[[""the picture that changed our view of the un...",[[]],"[[""the picture that changed our view of the un...","[[""changed""]]",annoyance,annoyance


In [13]:
# Manually process data with strange escape characters/mismatching spans to headlines and running the 'nones' list 10 code blocks below
# print(df[df.cause.str.contains('\\\\')].index)
weird_escapes_l = [382,587,815,903,1339,1601,1602,2187,2227,2549,2675,3446,3553,3560,4453,4529,4922,4938]
df_proper = df.drop(weird_escapes_l)
df_proper = df_proper[['headline', 'dominant_emotion', 'cause']]
cause_proper_list = df_proper.cause.apply(ast.literal_eval)
df_proper['cause_span'] = cause_proper_list.apply(lambda x: x[0][0] if x[0] else '')
df_proper

,headline,dominant_emotion,cause,cause_span
0,Cops in One Village Have Been Convicted of 70 ...,anger,"[[""cops in one village have been convicted of ...",cops in one village have been convicted of 70 ...
1,DIY penis enlargements are a 'nationwide probl...,negative_surprise,"[[""diy penis enlargements are a 'nationwide pr...",diy penis enlargements are a 'nationwide problem'
2,Dam breaking: New Epstein accuser comes forward,anger,"[[""new epstein accuser comes forward""]]",new epstein accuser comes forward
3,David Beckham gets six-month driving ban for u...,negative_surprise,"[[""gets six-month driving ban for using phone ...",gets six-month driving ban for using phone at ...
4,Dead sea turtle found with spear through head ...,sadness,"[[""dead sea turtle found with spear through he...",dead sea turtle found with spear through head ...
...,...,...,...,...
4995,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,negative_surprise,"[[""on her gun violence claim and only makes he...",on her gun violence claim and only makes herse...
4996,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,negative_surprise,"[[""‘triad’ thugs use clubs to punish hong kong...",‘triad’ thugs use clubs to punish hong kong’s ...
4997,'For real': High school football season begins...,positive_anticipation_including_optimism,"[[""season begins this week with expectations, ...","season begins this week with expectations, exc..."
4998,100 years on: the picture that changed our vie...,positive_surprise,"[[""the picture that changed our view of the un...",the picture that changed our view of the universe


In [14]:
# Manually adjust cause spans (there weren't too many of these to pose a time concern)
df_improper = df.iloc[weird_escapes_l]
df_improper = df_improper[['headline', 'dominant_emotion', 'cause']]
cause_improper_list = [
  '"Immoral or Scandalous" Trademarks—But "Vulgar or Profane" Marks, Maybe',
  'full-term abortion clinic injures woman during botched abortion – report',
  "believed it was 'the year of the black woman'",
  'War Crimes and Were Told to "Let It Go"',
  'Andrew Cockburn Looks at His "Disastrous Legislative Legacy"',
  'Attacks on Rep. Omar Rooted in Belief "America Is for White People"',
  'The Weakness of the "Whiteness" Literature',
  '"criminal act"',
  '"Racism and White Supremacy"',
  'north korea is waiting for trump to blink--or leave office',
  'About the US Problem with "Fake News"',
  'Claim That Green New Deal Is "Elitist"',
  "doesn’t understand culture",
  "About Their Upcoming 10-Day Holiday",
  "Supergroup Consisting of Da Candy x Justin Prime x Onderkoffer featuring Jackie's Boy & Lil Eddie Releases, \"Fever,\" June 14th, 2019",
  'are all releasing fixes for zombieload, a scary security flaw in intel chips that researchers just discovered (intc)',
  "'Cool that Michelle Obama knows I exist'",
  '"Created a Fake Crisis & Has Announced a Fake Solution" with Mexico'
  ]
df_improper['cause_span'] = cause_improper_list
df_improper

,headline,dominant_emotion,cause,cause_span
382,"Congress May Not Bar Registration of ""Immoral ...",disgust,"[[""immoral or scandalous\\""\\"" trademarks—but ...","""Immoral or Scandalous"" Trademarks—But ""Vulgar..."
587,Full-Term Abortion Clinic Injures Woman During...,anger,"[[""full-term abortion clinic injures woman dur...",full-term abortion clinic injures woman during...
815,"Rep. Clyburn disappointed by Kamala Harris, be...",negative_surprise,"[[""believed it was \\""the year of the black wo...",believed it was 'the year of the black woman'
903,Navy SEALs Tried for Months to Report Superior...,annoyance,"[[""war crimes and were told to \\""\\""let it go...","War Crimes and Were Told to ""Let It Go"""
1339,"As Joe Biden Hints at Presidential Run, Andrew...",shame,"[[""andrew cockburn looks at his \\""\\""disastro...","Andrew Cockburn Looks at His ""Disastrous Legis..."
1601,Anti-Racist Historian: Attacks on Rep. Omar Ro...,disgust,"[[""attacks on rep. omar rooted in belief \\""\\...","Attacks on Rep. Omar Rooted in Belief ""America..."
1602,"The Weakness of the ""Whiteness"" Literature",positive_surprise,"[[""the weakness of the \\""\\""whiteness\\""\\"" l...","The Weakness of the ""Whiteness"" Literature"
2187,State coroner in Greece says woman believed to...,sadness,"[[""\\""\\""criminal act\\""\\""\\""""]]","""criminal act"""
2227,Baltimore Residents: Trump's Attacks on the Ci...,anger,"[[""racism and white supremacy"",""attacks on the...","""Racism and White Supremacy"""
2549,North Korea Is Waiting for Trump to Blink--or ...,positive_anticipation_including_optimism,"[[""north korea is waiting for trump to blink, ...",north korea is waiting for trump to blink--or ...


In [15]:
df_clean = pd.concat([df_proper, df_improper])
df_clean = df_clean.sort_index()
df_clean

,headline,dominant_emotion,cause,cause_span
0,Cops in One Village Have Been Convicted of 70 ...,anger,"[[""cops in one village have been convicted of ...",cops in one village have been convicted of 70 ...
1,DIY penis enlargements are a 'nationwide probl...,negative_surprise,"[[""diy penis enlargements are a 'nationwide pr...",diy penis enlargements are a 'nationwide problem'
2,Dam breaking: New Epstein accuser comes forward,anger,"[[""new epstein accuser comes forward""]]",new epstein accuser comes forward
3,David Beckham gets six-month driving ban for u...,negative_surprise,"[[""gets six-month driving ban for using phone ...",gets six-month driving ban for using phone at ...
4,Dead sea turtle found with spear through head ...,sadness,"[[""dead sea turtle found with spear through he...",dead sea turtle found with spear through head ...
...,...,...,...,...
4995,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,negative_surprise,"[[""on her gun violence claim and only makes he...",on her gun violence claim and only makes herse...
4996,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,negative_surprise,"[[""‘triad’ thugs use clubs to punish hong kong...",‘triad’ thugs use clubs to punish hong kong’s ...
4997,'For real': High school football season begins...,positive_anticipation_including_optimism,"[[""season begins this week with expectations, ...","season begins this week with expectations, exc..."
4998,100 years on: the picture that changed our vie...,positive_surprise,"[[""the picture that changed our view of the un...",the picture that changed our view of the universe


In [16]:
# Remove blank gold causes. Convert to lower case to avoid discrepancies in BERT tokenization.
df_causes = df_clean.loc[df_clean.cause != '[[]]']
df_causes.headline = df_causes.headline.apply(str.lower)
df_causes.cause_span = df_causes.cause_span.apply(str.lower)
df_causes

<ipython-input-16-9b23c7edcf2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_causes.headline = df_causes.headline.apply(str.lower)
<ipython-input-16-9b23c7edcf2a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_causes.cause_span = df_causes.cause_span.apply(str.lower)


,headline,dominant_emotion,cause,cause_span
0,cops in one village have been convicted of 70 ...,anger,"[[""cops in one village have been convicted of ...",cops in one village have been convicted of 70 ...
1,diy penis enlargements are a 'nationwide probl...,negative_surprise,"[[""diy penis enlargements are a 'nationwide pr...",diy penis enlargements are a 'nationwide problem'
2,dam breaking: new epstein accuser comes forward,anger,"[[""new epstein accuser comes forward""]]",new epstein accuser comes forward
3,david beckham gets six-month driving ban for u...,negative_surprise,"[[""gets six-month driving ban for using phone ...",gets six-month driving ban for using phone at ...
4,dead sea turtle found with spear through head ...,sadness,"[[""dead sea turtle found with spear through he...",dead sea turtle found with spear through head ...
...,...,...,...,...
4995,‘now do abortion. we’ll wait.’ ilhan omar doub...,negative_surprise,"[[""on her gun violence claim and only makes he...",on her gun violence claim and only makes herse...
4996,‘triad’ thugs use clubs to punish hong kong’s ...,negative_surprise,"[[""‘triad’ thugs use clubs to punish hong kong...",‘triad’ thugs use clubs to punish hong kong’s ...
4997,'for real': high school football season begins...,positive_anticipation_including_optimism,"[[""season begins this week with expectations, ...","season begins this week with expectations, exc..."
4998,100 years on: the picture that changed our vie...,positive_surprise,"[[""the picture that changed our view of the un...",the picture that changed our view of the universe


In [17]:
df_causes.dominant_emotion.value_counts()

negative_surprise                            871
positive_surprise                            562
anger                                        450
fear                                         408
sadness                                      376
disgust                                      375
annoyance                                    341
negative_anticipation_including_pessimism    315
positive_anticipation_including_optimism     310
joy                                          260
guilt                                        171
trust                                        119
shame                                        118
pride                                         65
love_including_like                           58
Name: dominant_emotion, dtype: int64

# Details of prediction process:
The start and end indices of Bhat and Modi's model are IN RELATION TO THE NUMBER OF BERT TOKENS. These MUST stay consistent with the tokenizer splitting (i.e., it must take into account suffixes like '-est', '-ed', etc.). Say the model predicts start index i_s and end index i_e. We generate these indices in relation to the bert tokenizer so that we can simply grab the span via
spanbert_tokenizer.convert_tokens_to_string(spanbert_tokenizer.tokenize(annotation[i_s:i_e+1])) and compute F1 scores.

In [18]:
cause_model_name = 'mrm8488/spanbert-finetuned-squadv2'

spanbert_tokenizer = BertTokenizer.from_pretrained(cause_model_name)
spanbert_model = TFBertModel.from_pretrained(cause_model_name, output_hidden_states = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
# Get tokenized headlines.
headline_tokens = spanbert_tokenizer(list(df_causes.headline))['input_ids']

In [20]:
# Get tokenized cause spans. Remove start and end token.
cause_tokens = spanbert_tokenizer(list(df_causes.cause_span))['input_ids']
cause_tokens = [c[1:-1] for c in cause_tokens]

In [21]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in [i for i,e in enumerate(l) if e==sl[0]]:
        if l[ind:ind+sll]==sl:
            return (ind,ind+sll-1)

In [22]:
start_end_indices = []
for i in range(len(headline_tokens)):
  start_end_indices.append(find_sub_list(cause_tokens[i], headline_tokens[i]))

In [23]:
nones = [i for (i, e) in enumerate(start_end_indices) if e is None]
nones

[]

In [24]:
df_causes['start_index'] = [i[0] for i in start_end_indices]
df_causes['end_index'] = [i[1] for i in start_end_indices]

<ipython-input-24-757707e8819f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_causes['start_index'] = [i[0] for i in start_end_indices]
<ipython-input-24-757707e8819f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_causes['end_index'] = [i[1] for i in start_end_indices]


In [31]:
sorted(np.unique(df_causes.dominant_emotion))

['anger',
 'annoyance',
 'disgust',
 'fear',
 'guilt',
 'joy',
 'love_including_like',
 'negative_anticipation_including_pessimism',
 'negative_surprise',
 'positive_anticipation_including_optimism',
 'positive_surprise',
 'pride',
 'sadness',
 'shame',
 'trust']

In [32]:
df_causes['emotion_label'] = df_causes.dominant_emotion.map({'anger':0,
 'annoyance':1,
 'disgust':2,
 'fear':3,
 'guilt':4,
 'joy':5,
 'love_including_like':6,
 'negative_anticipation_including_pessimism':7,
 'negative_surprise':8,
 'positive_anticipation_including_optimism':9,
 'positive_surprise':10,
 'pride':11,
 'sadness':12,
 'shame':13,
 'trust':14})

<ipython-input-32-7e14e582cdd1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_causes['emotion_label'] = df_causes.dominant_emotion.map({'anger':0,


In [33]:
df_causes

,headline,dominant_emotion,cause,cause_span,start_index,end_index,emotion_label
0,cops in one village have been convicted of 70 ...,anger,"[[""cops in one village have been convicted of ...",cops in one village have been convicted of 70 ...,1,10,0
1,diy penis enlargements are a 'nationwide probl...,negative_surprise,"[[""diy penis enlargements are a 'nationwide pr...",diy penis enlargements are a 'nationwide problem',1,13,8
2,dam breaking: new epstein accuser comes forward,anger,"[[""new epstein accuser comes forward""]]",new epstein accuser comes forward,4,12,0
3,david beckham gets six-month driving ban for u...,negative_surprise,"[[""gets six-month driving ban for using phone ...",gets six-month driving ban for using phone at ...,6,16,8
4,dead sea turtle found with spear through head ...,sadness,"[[""dead sea turtle found with spear through he...",dead sea turtle found with spear through head ...,1,14,12
...,...,...,...,...,...,...,...
4995,‘now do abortion. we’ll wait.’ ilhan omar doub...,negative_surprise,"[[""on her gun violence claim and only makes he...",on her gun violence claim and only makes herse...,18,29,8
4996,‘triad’ thugs use clubs to punish hong kong’s ...,negative_surprise,"[[""‘triad’ thugs use clubs to punish hong kong...",‘triad’ thugs use clubs to punish hong kong’s ...,1,20,8
4997,'for real': high school football season begins...,positive_anticipation_including_optimism,"[[""season begins this week with expectations, ...","season begins this week with expectations, exc...",9,16,9
4998,100 years on: the picture that changed our vie...,positive_surprise,"[[""the picture that changed our view of the un...",the picture that changed our view of the universe,5,13,10


In [34]:
# Test string to check tokenization results
test_string = "diy penis enlargements are a 'nationwide problem'"
print(spanbert_tokenizer(test_string))
print(spanbert_tokenizer.tokenize(test_string))

{'input_ids': [101, 4267, 1183, 21504, 4035, 5815, 20512, 1116, 1132, 170, 112, 9708, 2463, 112, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['di', '##y', 'penis', 'en', '##lar', '##gement', '##s', 'are', 'a', "'", 'nationwide', 'problem', "'"]


In [35]:
# Train, dev, and test split
df_causes_shuffled = df_causes.sample(frac=1)
gne_train_preprocessed = df_causes_shuffled[:3072]
gne_valid_preprocessed = df_causes_shuffled[3072:3839]
gne_test_preprocessed = df_causes_shuffled[3839:]

In [37]:
from google.colab import files
gne_train_preprocessed.to_csv('gne_train_preprocessed.csv')
files.download('gne_train_preprocessed.csv')
gne_valid_preprocessed.to_csv('gne_valid_preprocessed.csv')
files.download('gne_valid_preprocessed.csv')
gne_test_preprocessed.to_csv('gne_test_preprocessed.csv')
files.download('gne_test_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>